## Getting Started with Predicate Searches

### MCTS-based searches

The ```BasicMCTSSearch.kt``` class defines a search method that makes use of a simplified MCTS algorithm to look for predicates that optimize the value of the given evaluation function. Internally, this class defines the search problem as a single-player game where the state (```GrammarProductionState.kt```) corresponds to a list of symbols derived from the search grammar, some of which are terminal (i.e. cannot be expanded any further), some not. These latter non-terminal symbols are associated with grammar rules and can thus be expanded in all the different ways that their rule allows.

For example, if we were to use the ```ifThenGrammar.bnf``` as our search grammar, given by:

```
<expr> ::=  G(!( <literal> ) "|" ( <literal> )) | G(!( <literal> ) "|" F ( <literal> )) | ( <expr> ) & ( <expr> )
```

the initial game state would be ```<expr>```, which consists of a single non-terminal symbol that can be expanded in 3 different ways following the productions of its grammar rule. Each of these production choices is treated as a game action when performing MCTS, and leads to a successor state where the expanded symbol is replaced by the ones of the chosen production. For instance, if we take the action that expands ```<expr> ::= ( <expr> ) & ( <expr> )```, the new game state would simply be represented by ```( <expr> ) & ( <expr> )```, and now we would have a total of 6 possible actions to take from there: 3 for each of the ```<expr>``` symbols added to the base expression.

Let's replicate this behavior by manually declaring an initial grammar production state from the ```<expr>``` symbol and the ```ifThenGrammar.bnf``` grammar, and then observing how the state representation and possible actions evolve when following the steps from the description above. 

In [1]:
import org.moeaframework.util.grammar.Symbol
import predcompiler.compilation.mcts.GrammarProductionState
import predcompiler.compilation.AbstractPredicateSearch
import predcompiler.compilation.mcts.buildResultString
import predcompiler.compilation.mcts.possibleActions

val grammar = AbstractPredicateSearch.readGrammar("../main/resources/grammars/ifThenGrammar.bnf")
val symbolSequence = listOf(Symbol("expr", false))
// intialize a state from the base grammar and the initial symbol sequence containing a single <expr> token.
val initialGameState = GrammarProductionState(grammar, symbolSequence)
// display a string representation of the current sybol sequence
println("Current state representation: " + initialGameState.buildResultString(ignoreNonTerminals = false))
// print available actions
println("Available Actions from current state:")
val possibleActions = initialGameState.possibleActions()
for (action in possibleActions) {
    println("\t" + action)
}

Current state representation: expr
Available Actions from current state:
	GrammarProductionAction(symbolToExpandIndex=0, chosenProductionIndex=0)
	GrammarProductionAction(symbolToExpandIndex=0, chosenProductionIndex=1)
	GrammarProductionAction(symbolToExpandIndex=0, chosenProductionIndex=2)


Note how each grammar production action is defined by two indices: one to represent which of the available symbols we wish to expand, and another to select the production to apply from the list of available productions in the symbol's grammar rule. Let's see what happens now if we apply the third action to the initial state.

In [2]:
import predcompiler.compilation.mcts.applyProduction

// produce a successor state by applying the next action to the current one
val nextState = initialGameState.applyProduction(possibleActions[2])
println("Next state representation: " + nextState.buildResultString(ignoreNonTerminals = false))
// print available actions
println("Available Actions from next state:")
val nextPossibleActions = nextState.possibleActions()
for (action in nextPossibleActions) {
    println("\t" + action)
}

Next state representation: (expr)&(expr)
Available Actions from next state:
	GrammarProductionAction(symbolToExpandIndex=1, chosenProductionIndex=0)
	GrammarProductionAction(symbolToExpandIndex=1, chosenProductionIndex=1)
	GrammarProductionAction(symbolToExpandIndex=1, chosenProductionIndex=2)
	GrammarProductionAction(symbolToExpandIndex=5, chosenProductionIndex=0)
	GrammarProductionAction(symbolToExpandIndex=5, chosenProductionIndex=1)
	GrammarProductionAction(symbolToExpandIndex=5, chosenProductionIndex=2)


Observe how there are now 6 possible actions for the two ```expr``` symbols at positions 1 and 5, respectively. A game state is considered terminal when all of its symbols are terminal and thus cannot be expanded anymore.

We will now look into how to perform a MCTS on the same grammar to find predicates that (attempt to) maximize their fitness according to a given evaluation heuristic. An example snippet on how to do this for the ping pong example is shown below.

In [3]:
import predcompiler.compilation.io.loadClassFromFile

val configFile = "../main/resources/experiments/ping_pong_mcts_sample.json"
val search = loadClassFromFile<AbstractPredicateSearch>(configFile)

while (search.isStillRunning) {
    search.step()
    search.printStepResults()
}

G(!(literal)|F(literal))
G(!(maybe-a&maybe-b)|F(literal))
G(!(maybe-a&maybe-b)|F(maybe-a&maybe-b))
G(!(maybe-a&1)|F(maybe-a&maybe-b))
G(!(maybe-a&1)|F(maybe-a&!b))
G(!(maybe-a&1)|F(a&!b))
G(!(1&1)|F(a&!b))
-------------------------------
Step 1
-------------------------------
Best Fitness Value: 100.0
    G(!(1&1)|F(a&!b))
-------------------------------

